## Multi Agent Customer Support Automation

In [1]:
pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew

In [4]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'

In [ ]:
#Support Agent
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

In [ ]:
#Support Quality Assurance Agent
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

* Role Playing: Both agents have been given a role, goal and backstory.

* Focus: Both agents have been prompted to get into the character of the roles they are playing.

 * Cooperation: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

In [8]:
from crewai_tools import SerperDevTool, \
                        ScrapeWebsiteTool, \
                        WebsiteSearchTool

In [ ]:
#Tools for scraping from the website
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

In [10]:
# Inquiry Task
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool], #Giving the tools during task or agent is allowed
    agent=support_agent,
)

In [ ]:
#Quality Assurance Review Task
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),                                                                  #No tools used
    agent=support_quality_assurance_agent,
)

In [ ]:
##  CREW
crew = Crew(
    agents=[support_agent, support_quality_assurance_agent],
    tasks=[inquiry_resolution, quality_assurance_review],
    verbose=2,
    memory=True  #Access the memory
)

2025-03-25 17:09:46,268 - 6286569472 - __init__.py-__init__:362 - ERROR: Exception while exporting Span batch.
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/util/connection.py", line 95, in create_connection
    raise err
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/util/connection.py", line 85, in create_connection
    sock.connect(sa)
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py", line 716, in urlopen
    httplib_response = self._make_request(

In [16]:
inputs = {
    "customer": "Stevens Institute of Technology",  #Interpolation variables customer, person and inquiry
    "person": "Amrutha",
    "inquiry": "I need help with setting up a Crew "
                "and kicking it off, specifically "
                "how can I add memory to my crew? "
                "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: Stevens Institute of Technology just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Amrutha from Stevens Institute of Technology is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
To provide the best possible support to Amrutha from Stevens Institute of Technology regarding setting up a Crew and adding memory to it, I need to gather detailed information from the specific documentation available.

Action: Read website content
Action Input: {"url": "https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off"}
 

Introduction - CrewAICrewAI home pageSearch CrewAI docscrewAIInc/crewAIcrewAIInc/crewAISear

In [17]:
from IPython.display import Markdown
Markdown(result)

Hello Amrutha,

Thank you for reaching out to us with your question about setting up a Crew and adding memory to it at CrewAI. I'm here to help you with that!

From the documentation I've reviewed, CrewAI is a powerful Python framework designed to create autonomous AI agent teams, known as Crews, that work collaboratively to tackle complex tasks. Each agent within a Crew has defined roles and can utilize specific tools to achieve their objectives.

Regarding adding memory to your Crew, while the specific details weren't explicitly covered in the documentation I accessed, typically, in AI frameworks like CrewAI, memory involves the ability of agents to retain and recall information across tasks or interactions. This can be implemented by configuring the agents with state management capabilities or by integrating external data storage solutions where agents can store and retrieve information as needed.

To set up a Crew and kick it off with memory capabilities, you would generally follow these steps:

1. **Define the Structure of Your Crew**: Determine the roles and tasks for each agent in your Crew. Each agent should have a clear purpose and access to the necessary tools.

2. **Configure Memory Capabilities**: Implement memory by ensuring your agents can maintain state or connect to external databases. This might involve customizing the agents with specific tools that allow for data persistence.

3. **Kick Off the Crew**: Use the CrewAI framework to initiate your Crew, ensuring that your agents are ready to collaborate, share insights, and manage tasks effectively.

4. **Test and Iterate**: Once your Crew is operational, monitor their performance and make adjustments to improve their memory usage and overall efficiency.

For more detailed guidance, I recommend visiting the documentation at [docs.crewai.com](https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off) or reaching out to our support team directly for personalized assistance.

I'm here to ensure you have all the support you need, so please feel free to reach out with any more questions or if there's anything else I can do to assist you!

Best regards,
[Your Name]
Senior Support Representative at CrewAI